In [ ]:
# from mp_workshop.mpcontribs import data
from mpcontribs.client import Client

In [ ]:
name = 'sandbox' # this should be your project
client = Client() # uses MPCONTRIBS_API_KEY envvar

**Retrieve and update project info**

In [ ]:
client.projects.update_entry(pk=name, project={
#     'unique_identifiers': False,
#     'references': [
#         {'label': 'Docs', 'url': 'https://mpcontribs.org'},
#         {'label': 'Source', 'url': 'https://refractiveindex.info'}
#     ]
    'columns': []
}).result()

In [ ]:
client.get_project(name).pretty()

**Create contributions**

In [ ]:
# key = "bogdan"
contributions = []
for _, contribs in data.items():
    for contrib in contribs:
        contrib["project"] = name
        contrib["is_public"] = True
        contributions.append(contrib)
    
len(contributions)

**Submit contributions**

In [ ]:
contributions = [
    {
        'identifier': 'AuCl',
        'data': {
            'compound': 'AuCl',
            'phase': 'rhomb',
            'reference': 'https://doi.org/...',
            'T': '298 K',
            'S': '0.0858±0.0042 kJ/deg/mol'
            'polynomial': {
                'D': 0
            },
            'ΔHᶠ': '-36.4±2.1 kJ/mol'
        },
        'project': name,
        'is_public': False
    }
]

In [21]:
client.delete_contributions(name)
# client.submit_contributions(contributions, per_page=100)
# client.contributions.create_entries(contributions=contributions).result()

In [9]:
client.contributions.get_entries(project=name).result()

{'data': [], 'has_more': False, 'total_count': 0, 'total_pages': 0}

**Query contributions**

In [ ]:
resp = client.contributions.get_entries(
    project=name, identifier="mp-1224792", _fields=["id"]
).result()
cids = [d["id"] for d in resp["data"]]
contrib = client.get_contribution(cids[0])
contrib.pretty()

In [ ]:
tid = contrib["tables"][0]["id"]
client.get_table(tid)#.plot()

In [ ]:
query = {
    "project": name,
    "formula__contains": "Li",
    "data__type__contains": "f4",
    "data__coefficients__c1__value__gte": 4.93e-3,
    "_order_by": "data__coefficients__c1__value",
    "order": "desc",
    "_fields": [
        "id", "identifier", "formula",
        "data.type", "data.coefficients.c0.value",
        "data.coefficients.c1.value"
    ]
}
client.contributions.get_entries(**query).result()

**Delete contributions**

In [ ]:
client.contributions.delete_entries(id__in=cids).result()
# or in your own project use `client.delete_contributions(name)`
# to delete all contributions using pagination